In [ ]:
#Base Python imports
import sys
import os

In [ ]:
##Get the location of raw_pickles, assuming you run this notebook in xrb-sens-datashare/raw_pickles
import os
cur_data_root = os.getcwd()
print('Using raw_pickles data root:\n   {}'.format(cur_data_root))
#'/home/ajacobs/Reporoot/xrb-sens-datashare/raw_pickles'

In [ ]:
#Derive analysis and kepler locations from root
package_root = '{}/../analysis_packages'.format(cur_data_root)
kepler_package_root = '{}/../kepler_python_packages'.format(cur_data_root)

#Insert kepler python packages into the python path
sys.path.insert(0, '{}/python_scripts'.format(kepler_package_root))

In [ ]:
#Provide analysis, data abstraction, and reduction modules as well as modules
#that understand Kepler's data formats
#NOTE: This is very hacky and brittle. Should be improved in next development iteration.
import re
from pickle import dump, load
from glob import glob
from os.path import basename, join
from random import sample
from scipy.constants import physical_constants

#import grid_setup
#from model_analysis import ModelAnalysis
from lcdata import LCData
from kepdata import KepData
from kepdump import KepDump
from isoplot import IsoPlot
from nucplot import IsoPlot as NucPlot
import bdat
import ionmap
from isotope import el2z

With dependencies provided, reading up to the entire raw data set is as easy as
manipulating a nested Python dictionary full of performant numpy objects as
well as self-documenting labels/descriptions.

Below, we explore the dictionary at a high level

In [ ]:
#Below is a reference example of leading in the pickle, doing some checks, and
#highlighting some of the detailed data available.

#Load in the raw data
#with open('4u1820ZJ_100x_grid_data.Nov18.pk', 'rb') as f:
with open('gs1826_10x_grid_data.Nov5.pk', 'rb') as f:
    grid_data = load(f)

gd = grid_data

#Do a basic check of the data
#summarize the grid properties
print("Grid label:       {}".format(gd['grid_label']))
print("Grid description: {}".format(gd['grid_desc']))
print("X:      {}".format(gd['x']))
print("Z(n14): {}".format(gd['z']))
print("Q_b:    {} MeV / u".format(gd['qb']))
print("Eddington fraction: {}".format(gd['eddf']))
print("xi: {}".format(gd['xi']))
print("Accretion Lum: {} erg / s".format(gd['acc_lum']))
print("Accretion Rate: {} M_sol / yr".format(gd['acc_rate']))
print("surface gravity, g: {} cm / s^2".format(gd['gee']))
print("# of variations (may be off by 1, see source): {}".format(len(gd['vary_list'])-1)) #-1 because we currently put fake variation in first to gen the baseline
print("# of models: {}".format(len(gd['model_data'])))
print("")

Data on the entire grid is at the first level of the nested dictionary.  Specifically, they are:
    
```
grid_data{
    'grid_label': simple string labeling this grid of data,
    'grid_desc':  short human-readable description of the grid,
    'x':          hydrogen mass fraction of the baseline bursting model,
    'z':          metalicity of baseline model,
    'qb':         base heating in MeV/u (has dependence on accretion),
    'eddf':       Eddington fraction of the luminosity,
    'xi':         geometric factor that's not used, just set to 1 for this study,
    'acc_lum':    accretion luminosity, the luminosity generated at base by accretion,
    'acc_rate':   accretion rate,
    'gee':        local gravity g,
    'vary_list':  a list of tuples descripting this grid's variations as ('Abc', X) 
                  where Abc references reaction form A(b,c)D and X is the factor that reaction was scaled by,
    'model_data': the bulk of the model data is here, we will dive into it later,
    'models_to_debug': a list of models experiencing issues,
}
```
Concrete examples can be seen in the next code cell  

In [ ]:
for key in gd.keys():
    if key == 'model_data':
        continue
    print('{} --> {}'.format(key, gd[key]))


The detailed data for each model can be found in `grid_data['model_data']` and is keyed based on the model's reaction variations.

For example, `grid_data['model_data']['dag0.1']` contains data for the baseline model (GS1826 in this case) with a variation of the d(alpha, gamma) reaction down by a factor of 10 (hence 0.1).

`grid_data['model_data']['o15ag10.0']` contains data for O15 (alpha, gamma) being varied up by a factor of 10.

Model labels generally follow this form.  Below, we print out a list of all model labels.  Note that on occasion special labels are made, like 'base'.

In [ ]:
model_count = len(gd['model_data'])
print('There are a total of {} models in model_data\n'.format(model_count))
labels = (label for label in gd['model_data'].keys())

for label in labels:
    print(label)

Each model has associated data, as we see here

In [ ]:
print(gd['model_data']['o15ag10.0'].keys())
print('\n')

#Overview and bulk data
for key in list(gd['model_data']['o15ag10.0'].keys())[0:5]:
    print('{} --> {}'.format(key, gd['model_data']['o15ag10.0'][key]))
key='kepler_label' #this is a simple short label used by kepler to keep track of the model
print('{} --> {}'.format(key, gd['model_data']['o15ag10.0'][key]))
    
    
    
#Raw full lightcurve data
print('raw_lc_time --> <numpy array of raw lightcurve time in seconds>')
print('raw_lc_lum --> <numpy array of raw lightcurve luminosity in erg/s>')
print('raw_lc_rad --> <numpy array of raw lightcurve photosphere radius in cm>')

#Burst data in which the above lightcurve is broken up into distinct bursts (`nbursts` tells you how many there are)
print('burst_times --> <a list of `nburst` numpys arrays containing that burst time, with t=0s being the peak time>')
print('burst_lums  --> <a list of `nburst` numpys arrays containing that burst luminosity L(t), with t coming from corresponding burst_times>')
print('alc_arr     --> <a list of [time, lum, rad] numpy arrays representing the average of all bursts>')

#Detailed profile (ash!) data is available through kepler's dump files:
print(gd['model_data']['o15ag10.0']['dump_data'].keys())

#More data is available for each dumpfile, but some pickles (like this one) only have initial bulk data (the model's iteration ncyc, index of ash base jblo, path, etc) and will need the full ash data added once brought in from iCER:
print(gd['model_data']['o15ag10.0']['dump_data']['gs18#32000'].keys())